<a href="https://colab.research.google.com/github/WorldExplored/Medical_AI_Prompter/blob/main/Medical_AI_Prompter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install replicate
!pip install PyPDF2

In [ ]:
import PyPDF2
import replicate
from replicate.client import Client

# PDF extraction
file = input("PDF File Directory: ")
pdf_file = open(file, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)
text = ''

for page_num in range(num_pages):
    page = pdf_reader.pages[page_num]
    page_text = page.extract_text()
    text += page_text

pdf_file.close()

# Initialize the Replicate Client with your API token
client = Client(api_token="r8_YlX943jw2pCjXrHLGoZEk0E6m3uO4zP07oTAt")

# Function to interact with the LLM
def ask_medical_llm(client, question, chat_log=None):
    if chat_log is None:
        chat_log = []

    chat_log.append({"role": "user", "content": question})
    prompt = "\n\n".join([f"{entry['role']}: {entry['content']}" for entry in chat_log])

    output = client.run(
        ref="meta/meta-llama-3-70b-instruct",
        input={"prompt": prompt}
    )

    answer = ''.join(output)
    chat_log.append({"role": "assistant", "content": answer})

    return answer, chat_log

# Chunk the extracted text to avoid exceeding the maximum input length
def chunk_text(text, chunk_size=250):
    for i in range(0, len(text), chunk_size):
        yield text[i:i + chunk_size]

# List of questions to be asked to the LLM
questions = {
    "Publication Year": "Provide the publication year.",
    "Disease": "Write down the disease condition from the methods/introduction section of the publication.",
    "Location": "Provide the study location information.",
    "Pt/Ptcpnt data": "Does the study include patient or participant data? (Yes/No)",
    "Relevant": "Is the publication relevant based on patient data for a rare disease condition and location in the US? (Yes/No)",
    "Reason": "If relevant, provide the reason.",
    "Study type": "Provide the type of study.",
    "Data Source": "Mention any data source like a database or a genebank.",
    "Notes": "Capture any useful notes for analysis.",
    "Age": "Is age reported? (Yes/No)",
    "Reported Sex/Gender": "Report the gender/sex reported.",
    "Race": "Is race reported? (Yes/No)",
    "Ethnicity": "Is ethnicity reported? (Yes/No)",
    "Language": "Is language reported? (Yes/No)",
    "Education": "Is education reported? (Yes/No)",
    "Employment": "Is employment reported? (Yes/No)",
    "Residence/practice location": "Provide residence/practice location if available.",
    "Relationship status": "Is relationship status reported? (Yes/No)",
    "Medical or Health Insurance": "Is medical or health insurance reported? (Yes/No)",
    "Social support": "Is social support reported? (Yes/No)",
    "Number of participants": "Provide the number of participants.",
    "Gender, % Female": "Provide the percentage of female participants. Find the EXPLICIT number.",
    "Gender, % Male": "Provide the percentage of male participants. Find the EXPLICIT number.",
    "Gender, % others": "Provide the percentage of other gender categories. Find the EXPLICIT number.",
    "% White": "Provide the percentage of White participants. Find the EXPLICIT number.",
    "% Black or African American": "Provide the percentage of Black or African American participants. Find the EXPLICIT number.",
    "% Asian": "Provide the percentage of Asian participants. Find the EXPLICIT number.",
    "% Native Hawaiian or Other Pacific Islander": "Provide the percentage of Native Hawaiian or Other Pacific Islander participants. Find the EXPLICIT number.",
    "% American Indian or Alaska Native": "Provide the percentage of American Indian or Alaska Native participants. Find the EXPLICIT number.",
    "% Others": "Provide the percentage of participants from other races. Find the EXPLICIT number.",
    "% Mixed race/multiracial": "Provide the percentage of mixed race/multiracial participants. Find the EXPLICIT number.",
    "RNR (race not reported/ unknown)": "Is race not reported or unknown? (Yes/No)",
    "% Hispanic or Latino": "Provide the percentage of Hispanic or Latino participants. Find the EXPLICIT number.",
    "% H/Other/Missing": "Provide the percentage of H/Other/Missing participants. Find the EXPLICIT number.",
    "% Non Hispanic": "Provide the percentage of Non-Hispanic participants. Find the EXPLICIT number.",
    "Non White%": "Provide the percentage of Non-White participants.",
    "Ethnicity not reported/ unknown": "Is ethnicity not reported or unknown? (Yes/No)",
    "College or more": "Is college or more education level reported? (Yes/No)",
    "Some college": "Is some college education level reported? (Yes/No)",
    "High school or less": "Is high school or less education level reported? (Yes/No)",
    "Employment": "Is employment status reported? (Yes/No)",
    "% Urban": "Provide the percentage of urban participants.",
    "% Rural": "Provide the percentage of rural participants.",
    "% Suburban": "Provide the percentage of suburban participants.",
    "% Location not reported": "Provide the percentage of participants with location not reported.",
    "Commercial/Private medical funding": "Provide the percentage of participants with commercial/private medical funding.",
    "Government medical funding": "Provide the percentage of participants with government medical funding.",
    "Self funding": "Provide the percentage of participants with self-funding.",
    "Other/Unknown": "Provide the percentage of participants with other/unknown funding.",
    "Married/ever married": "Is married/ever married status reported? (Yes/No)",
    "Single/never married": "Is single/never married status reported? (Yes/No)",
    "Unknown": "Is marital status unknown? (Yes/No)",
    "Disability allowance/DDSN data": "Is disability allowance/DDSN data reported? (Yes/No)",
    "Special education assistance": "Is special education assistance reported? (Yes/No)",
    "Food assistance": "Is food assistance reported? (Yes/No)"
}

# Function to fill in the blanks with answers from the LLM
def fill_in_blanks():
    chat_log = []
    for field, question in questions.items():
        question_answered = False
        for text_chunk in chunk_text(text):
            question_with_text = f"Based on the provided text, answer this: {question}\n\nText: {text_chunk}\n You MUST be concise as possible."
            if len(question_with_text) <= 8000:
                answer, chat_log = ask_medical_llm(client, question_with_text, chat_log)
                if answer.strip() != "NR":
                    questions[field] = answer.strip()
                    question_answered = True
                    break
        if not question_answered:
            questions[field] = "NR"
        print(f"Updated {field}: {questions[field]}")

fill_in_blanks()

# Interactive part for user queries
e = "blank"
while e:
    e = input("What is your query? ")
    if e.lower() in ["exit", "quit"]:
        break
    ans, log = ask_medical_llm(client, e, chat_log)
    print(ans)


PDF File Directory: u.pdf
Updated Publication Year: There is no publication year provided in the given text. The text appears to be a jumbled collection of characters and does not contain any meaningful information, including a publication year.
Updated Disease: There is no disease condition mentioned in the provided text. The text appears to be a jumbled collection of characters and does not contain any meaningful information.
Updated Location: There is no study location information provided in the given text.
Updated Pt/Ptcpnt data: No
Updated Relevant: No
Updated Reason: Not applicable, as the text is a jumbled collection of characters and does not contain any meaningful information.
Updated Study type: Not applicable, as the text is a jumbled collection of characters and does not contain any meaningful information.
Updated Data Source: None
Updated Notes: None, as the text is a jumbled collection of characters and does not contain any meaningful information.
Updated Age: No
Updated

KeyboardInterrupt: 